In [1]:
# Load AIEEE Data

In [2]:
%cd ../../

/home/souvic/mounted/btp/vahini/Name2Demographics


In [3]:
!nvidia-smi

Sat Mar 26 16:35:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.86       Driver Version: 470.86       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   33C    P0    33W / 250W |   2059MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:D8:00.0 Off |                    0 |
| N/A   

In [4]:
import sys  
sys.path.insert(0, 'Models/ERData/PreProcessing/')

from er_preprocess import *

In [5]:
er_df = pd.read_csv("Models/Data/ERData/er_data_with_majority_label.csv")

Load Models

In [6]:
#  other models
AIEEE_BERT = "Models/AIEEEGenderData/BERT/SavedModel/BERT"
CBSE_BERT = "Models/CBSEData/BERT/SavedModel/BERT"
ER_BERT = "Models/ERData/BERT/SavedModel/BERT"

In [7]:
%%capture
!pip install transformers[sentencepiece]
import transformers

In [8]:
from transformers import BertModel, AutoModelForSequenceClassification
from transformers import AutoTokenizer

Predict Gender

In [9]:
from transformers import pipeline
from sklearn.metrics import classification_report
import math
import torch

In [10]:
import os

In [23]:
def print_results(df):
#     print("hello")
    
#     # Extract tokenizer.
#     tokens = tokenizer(name, padding=True, truncation=True, return_tensors="pt")
#     output = aieee_model(**tokens)
#     predictions = torch.nn.functional.softmax(output.logits, dim=-1)
#     aieee_label = torch.argmax(predictions)
    
#     output = cbse_model(**tokens)
#     predictions = torch.nn.functional.softmax(output.logits, dim=-1)
#     cbse_label = torch.argmax(predictions)
    
#     print("end")
    

    with open("Models/ERData/er_data_test.txt", 'a') as f:
        f.write( f" started: {os.getpid()}")
        
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

    aieee_model = AutoModelForSequenceClassification.from_pretrained(AIEEE_BERT)
    cbse_model = AutoModelForSequenceClassification.from_pretrained(CBSE_BERT)
    
    with open("Models/ERData/er_data_test.txt", 'a') as f:
        f.write(f"Models loaded by {os.getpid()}\n")
        

    pipe = pipeline(task="text-classification",model=aieee_model, tokenizer=tokenizer)
    n = df.shape[0]
    predictions = []
    for i in range(math.ceil(n/1000)):
        df1 = df[i*1000:(i+1)*(1000)]
        onway = pipe(list(df1['Name']), batch_size=256)
        predictions = predictions + onway
        
    with open("Models/ERData/er_data_test.txt", 'a') as f:
        f.write(f"{n} are done by {os.getpid()}\n")
        
    labels = []
    for pred in predictions:
        if pred['label'] == 'LABEL_0':
            labels.append(0)
        else:
            labels.append(1)
    
    true = list(df['Gender'])
    aieee = (true, labels)
    
    pipe = pipeline(task="text-classification",model=cbse_model, tokenizer=tokenizer)
    n = df.shape[0]
    predictions = []
    for i in range(math.ceil(n/1000)):
        df1 = df[i*1000:(i+1)*(1000)]
        onway = pipe(list(df1['Name']), batch_size=256)
        predictions = predictions + onway
    labels = []
    for pred in predictions:
        if pred['label'] == 'LABEL_0':
            labels.append(0)
        else:
            labels.append(1)
    
    true = list(df['Gender'])
    cbse = (true, labels)
    
    return cbse, aieee

In [24]:
m = er_df.shape[0]

In [25]:
array = []
for i in range(72):
    df1 = er_df[i*100000:(i+1)*100000]
    array.append(df1)

In [26]:
from multiprocessing import Pool

if __name__ == '__main__':
    with Pool(15) as p:
        preds = p.map(print_results, list(array))

Process ForkPoolWorker-12:
Process ForkPoolWorker-17:
Process ForkPoolWorker-6:
Traceback (most recent call last):
  File "/home/souvic/.conda/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/souvic/.conda/envs/py38/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/souvic/.conda/envs/py38/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-14:
  File "/home/souvic/.conda/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Process ForkPoolWorker-18:
  File "/home/souvic/.conda/envs/py38/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/souvic/.conda/envs/py38/lib/python3.8/multiprocessing/pool.py", line 48, in maps

  File "/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
Process ForkPoolWorker-8:
  File "/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/transformers/models/bert/modeling_bert.py", line 1524, in forward
    outputs = self.bert(
  File "/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/transformers/models/bert/modeling_bert.py", line 990, in forward
    encoder_outputs = self.encoder(
  File "/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/transformers/models/bert/modeling_bert.py", line 582, in forward
    layer_outputs = layer_modu

  File "/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/transformers/pipelines/base.py", line 798, in _forward
    predictions = self.model(**inputs)[0].cpu()
  File "/home/souvic/.conda/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/souvic/.conda/envs/py38/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/souvic/.conda/envs/py38/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/transformers/models/bert/modeling_bert.py", line 582, in forward
    layer_outputs = layer_module(
  File "/home/souvic/.conda/envs/py38/lib/python3.8/multiprocessing/pool.py", line 48, in mapstar
    return list(map(*args))
  File "/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.

  File "/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/transformers/models/bert/modeling_bert.py", line 522, in feed_forward_chunk
    intermediate_output = self.intermediate(attention_output)
  File "/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/transformers/modeling_utils.py", line 2196, in apply_chunking_to_forward
    return forward_fn(*input_tensors)
  File "/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/transformers/models/bert/modeling_bert.py", line 425, in forward
    hidden_states = self.dense(hidden_states)
  File "/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/transformers/models/bert/modeling_bert.py", line 990, in forward
    encoder_outputs = self.encoder(
  File "/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/transformers/models/bert/modeling_bert.py", li

KeyboardInterrupt
  File "/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/transformers/pipelines/base.py", line 798, in _forward
    predictions = self.model(**inputs)[0].cpu()
  File "/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/transformers/models/bert/modeling_bert.py", line 990, in forward
    encoder_outputs = self.encoder(
  File "/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/transformers/models/bert/modeling_bert.py", line 1524, in forward
    outputs = self.bert(
  File "/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)


KeyboardInterrupt: 

In [18]:
er_df[:5]['Name']

0        patel vinaben kanubhai
1        bankim kantibhai patel
2    kamli mukesh rameshchandra
3       devisharan babulal yogi
4         anita devisharan yogi
Name: Name, dtype: object

In [ ]:
print_results("patel vinaben kanubhai")

In [11]:
er_df.shape[0]/1000

7149.489